In [ ]:
import pandas as pd
import numpy as np
import warnings
import sklearn
import seaborn as sns
import time
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine
from collections import Counter

from zipfile import ZipFile
from pathlib import Path
from tensorflow import keras
from tensorflow.keras import layers
warnings.filterwarnings("ignore")
from scipy.sparse.linalg import svds

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_sepeda = pd.read_csv('/content/sepeda.csv')
data_rating = pd.read_csv('/content/sepeda_ratings (1).csv')
data_sepeda.head()

,sepeda_id,nama_sepeda
0,1,Polygon Premier 4
1,2,Polygon Premier 5
2,3,Polygon Xtrada 5
3,4,Polygon Xtrada 6.1
4,5,Polygon Xtrada 7


In [ ]:
data_rating.head()

,user_id,sepeda_id,rating,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,1,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data_rating.drop('Unnamed: 3', axis = 1, inplace=True)
data_rating.drop('Unnamed: 4', axis = 1, inplace=True)
data_rating.drop('Unnamed: 5', axis = 1, inplace=True)
data_rating.drop('Unnamed: 6', axis = 1, inplace=True)
data_rating.drop('Unnamed: 7', axis = 1, inplace=True)
data_rating.drop('Unnamed: 8', axis = 1, inplace=True)
data_rating.drop('Unnamed: 9', axis = 1, inplace=True)
data_rating.drop('Unnamed: 10', axis = 1, inplace=True)
data_rating.drop('Unnamed: 11', axis = 1, inplace=True)
data_rating.drop('Unnamed: 12', axis = 1, inplace=True)
data_rating.head()

,user_id,sepeda_id,rating
0,1,1,2.0
1,2,1,3.0
2,3,1,NaN
3,4,1,1.0
4,5,1,2.0


In [ ]:
data_rating_sepeda = pd.merge(data_rating, data_sepeda, on = 'sepeda_id')
data_rating_sepeda.head()

,user_id,sepeda_id,rating,nama_sepeda
0,1,1,2.0,Polygon Premier 4
1,2,1,3.0,Polygon Premier 4
2,3,1,NaN,Polygon Premier 4
3,4,1,1.0,Polygon Premier 4
4,5,1,2.0,Polygon Premier 4


In [ ]:
sepeda_rating = data_rating_sepeda.pivot_table('rating', index = 'nama_sepeda', columns='user_id')
sepeda_rating.head(9)

user_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
nama_sepeda,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Polygon D7,4.0,4.0,5.0,NaN,3.0,5.0,3.0,3.0,4.0,NaN,3.0,2.0,3.0,1.0,NaN,3.0,3.0,2.0,NaN,3.0,3.0,NaN,2.0,2.0,4.0,NaN,3.0,3.0,4.0,NaN,4.0,4.0,NaN,4.0,2.0,NaN,5.0,NaN,NaN,4.0,3.0
Polygon Premier 4,2.0,3.0,NaN,1.0,2.0,NaN,2.0,NaN,2.0,NaN,NaN,NaN,2.0,2.0,1.0,1.0,NaN,1.0,2.0,2.0,NaN,4.0,2.0,1.0,2.0,1.0,NaN,1.0,2.0,2.0,2.0,NaN,2.0,2.0,1.0,NaN,2.0,1.0,2.0,NaN,2.0
Polygon Premier 5,3.0,NaN,3.0,2.0,NaN,2.0,2.0,2.0,NaN,3.0,4.0,NaN,3.0,NaN,1.0,1.0,NaN,1.0,NaN,2.0,NaN,NaN,2.0,1.0,2.0,NaN,1.0,1.0,2.0,3.0,NaN,2.0,3.0,NaN,2.0,3.0,NaN,1.0,2.0,2.0,2.0
Polygon Siskiu D5,3.0,NaN,3.0,3.0,2.0,4.0,NaN,2.0,1.0,1.0,NaN,1.0,3.0,1.0,2.0,NaN,1.0,1.0,2.0,NaN,3.0,3.0,2.0,NaN,3.0,3.0,1.0,NaN,2.0,2.0,3.0,NaN,1.0,4.0,NaN,2.0,5.0,3.0,NaN,4.0,3.0
Polygon Siskiu T7,5.0,5.0,3.0,5.0,NaN,3.0,4.0,3.0,2.0,NaN,3.0,NaN,NaN,1.0,3.0,5.0,NaN,2.0,3.0,NaN,5.0,4.0,3.0,NaN,3.0,2.0,NaN,4.0,2.0,4.0,NaN,4.0,3.0,NaN,2.0,2.0,5.0,NaN,3.0,5.0,NaN
Polygon Siskiu T8,5.0,NaN,4.0,5.0,NaN,4.0,NaN,5.0,NaN,NaN,4.0,2.0,5.0,2.0,4.0,5.0,5.0,2.0,3.0,5.0,5.0,4.0,5.0,4.0,5.0,4.0,NaN,5.0,5.0,5.0,4.0,NaN,3.0,5.0,3.0,4.0,5.0,5.0,4.0,5.0,3.0
Polygon Xtrada 5,4.0,3.0,NaN,2.0,3.0,4.0,NaN,2.0,3.0,NaN,1.0,2.0,4.0,5.0,2.0,NaN,2.0,2.0,4.0,2.0,2.0,5.0,2.0,NaN,3.0,2.0,2.0,NaN,3.0,4.0,3.0,NaN,3.0,2.0,NaN,3.0,3.0,NaN,2.0,3.0,2.0
Polygon Xtrada 6.1,4.0,4.0,NaN,3.0,4.0,5.0,3.0,3.0,NaN,4.0,2.0,2.0,3.0,NaN,4.0,1.0,3.0,3.0,NaN,3.0,2.0,NaN,NaN,2.0,NaN,2.0,4.0,2.0,3.0,4.0,NaN,3.0,3.0,3.0,4.0,3.0,3.0,2.0,NaN,3.0,4.0
Polygon Xtrada 7,5.0,NaN,4.0,4.0,4.0,5.0,NaN,NaN,NaN,NaN,3.0,5.0,5.0,5.0,4.0,1.0,NaN,5.0,5.0,3.0,NaN,4.0,5.0,3.0,3.0,2.0,4.0,2.0,NaN,4.0,4.0,3.0,4.0,3.0,4.0,5.0,NaN,2.0,3.0,3.0,4.0


In [ ]:
data_rating_sepeda.fillna(0, inplace = True)
data_rating_sepeda.head()

,user_id,sepeda_id,rating,nama_sepeda
0,1,1,2.0,Polygon Premier 4
1,2,1,3.0,Polygon Premier 4
2,3,1,0.0,Polygon Premier 4
3,4,1,1.0,Polygon Premier 4
4,5,1,2.0,Polygon Premier 4


In [ ]:
sepeda_rating.fillna(0, inplace = True)
sepeda_rating

user_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
nama_sepeda,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Polygon D7,4.0,4.0,5.0,0.0,3.0,5.0,3.0,3.0,4.0,0.0,3.0,2.0,3.0,1.0,0.0,3.0,3.0,2.0,0.0,3.0,3.0,0.0,2.0,2.0,4.0,0.0,3.0,3.0,4.0,0.0,4.0,4.0,0.0,4.0,2.0,0.0,5.0,0.0,0.0,4.0,3.0
Polygon Premier 4,2.0,3.0,0.0,1.0,2.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,2.0,2.0,1.0,1.0,0.0,1.0,2.0,2.0,0.0,4.0,2.0,1.0,2.0,1.0,0.0,1.0,2.0,2.0,2.0,0.0,2.0,2.0,1.0,0.0,2.0,1.0,2.0,0.0,2.0
Polygon Premier 5,3.0,0.0,3.0,2.0,0.0,2.0,2.0,2.0,0.0,3.0,4.0,0.0,3.0,0.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,2.0,1.0,2.0,0.0,1.0,1.0,2.0,3.0,0.0,2.0,3.0,0.0,2.0,3.0,0.0,1.0,2.0,2.0,2.0
Polygon Siskiu D5,3.0,0.0,3.0,3.0,2.0,4.0,0.0,2.0,1.0,1.0,0.0,1.0,3.0,1.0,2.0,0.0,1.0,1.0,2.0,0.0,3.0,3.0,2.0,0.0,3.0,3.0,1.0,0.0,2.0,2.0,3.0,0.0,1.0,4.0,0.0,2.0,5.0,3.0,0.0,4.0,3.0
Polygon Siskiu T7,5.0,5.0,3.0,5.0,0.0,3.0,4.0,3.0,2.0,0.0,3.0,0.0,0.0,1.0,3.0,5.0,0.0,2.0,3.0,0.0,5.0,4.0,3.0,0.0,3.0,2.0,0.0,4.0,2.0,4.0,0.0,4.0,3.0,0.0,2.0,2.0,5.0,0.0,3.0,5.0,0.0
Polygon Siskiu T8,5.0,0.0,4.0,5.0,0.0,4.0,0.0,5.0,0.0,0.0,4.0,2.0,5.0,2.0,4.0,5.0,5.0,2.0,3.0,5.0,5.0,4.0,5.0,4.0,5.0,4.0,0.0,5.0,5.0,5.0,4.0,0.0,3.0,5.0,3.0,4.0,5.0,5.0,4.0,5.0,3.0
Polygon Xtrada 5,4.0,3.0,0.0,2.0,3.0,4.0,0.0,2.0,3.0,0.0,1.0,2.0,4.0,5.0,2.0,0.0,2.0,2.0,4.0,2.0,2.0,5.0,2.0,0.0,3.0,2.0,2.0,0.0,3.0,4.0,3.0,0.0,3.0,2.0,0.0,3.0,3.0,0.0,2.0,3.0,2.0
Polygon Xtrada 6.1,4.0,4.0,0.0,3.0,4.0,5.0,3.0,3.0,0.0,4.0,2.0,2.0,3.0,0.0,4.0,1.0,3.0,3.0,0.0,3.0,2.0,0.0,0.0,2.0,0.0,2.0,4.0,2.0,3.0,4.0,0.0,3.0,3.0,3.0,4.0,3.0,3.0,2.0,0.0,3.0,4.0
Polygon Xtrada 7,5.0,0.0,4.0,4.0,4.0,5.0,0.0,0.0,0.0,0.0,3.0,5.0,5.0,5.0,4.0,1.0,0.0,5.0,5.0,3.0,0.0,4.0,5.0,3.0,3.0,2.0,4.0,2.0,0.0,4.0,4.0,3.0,4.0,3.0,4.0,5.0,0.0,2.0,3.0,3.0,4.0


# **Proses**

In [ ]:
n_users = data_rating_sepeda.user_id.unique().shape[0]
n_bikes = data_rating_sepeda.sepeda_id.unique().shape[0]
print ('Jumlah user = ' + str(n_users) + '| Jumlah sepeda = ' + str(n_bikes))

Jumlah user = 41| Jumlah sepeda = 9


In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data_rating_sepeda, test_size=0.25)

In [ ]:
train_data_matrix = np.zeros((n_users, n_bikes))
for line in train_data.itertuples():
  train_data_matrix[line[1]-1, line[2]-1] = line[3]

test_data_matrix = np.zeros((n_users, n_bikes))
for line in test_data.itertuples():
  test_data_matrix[line[1]-1, line[2]-1] = line[3]

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances
bikes_similarity = cosine_similarity(train_data_matrix.T)
bikes_similarity

array([[1.        , 0.39769745, 0.49953383, 0.51458159, 0.59467473,
        0.45571483, 0.41743401, 0.35387202, 0.58705829],
       [0.39769745, 1.        , 0.39732667, 0.61097746, 0.40758619,
        0.25890947, 0.35617215, 0.44471944, 0.4996292 ],
       [0.49953383, 0.39732667, 1.        , 0.61070903, 0.63097161,
        0.53539747, 0.4302823 , 0.59443009, 0.58551687],
       [0.51458159, 0.61097746, 0.61070903, 1.        , 0.61135682,
        0.48664521, 0.6422041 , 0.60213086, 0.64874792],
       [0.59467473, 0.40758619, 0.63097161, 0.61135682, 1.        ,
        0.57028506, 0.52691641, 0.48580009, 0.55032907],
       [0.45571483, 0.25890947, 0.53539747, 0.48664521, 0.57028506,
        1.        , 0.51057636, 0.55229329, 0.67927438],
       [0.41743401, 0.35617215, 0.4302823 , 0.6422041 , 0.52691641,
        0.51057636, 1.        , 0.71603048, 0.53039484],
       [0.35387202, 0.44471944, 0.59443009, 0.60213086, 0.48580009,
        0.55229329, 0.71603048, 1.        , 0.55126816],


In [ ]:
memory_based = pd.DataFrame(data = bikes_similarity, index = sepeda_rating.index, columns = sepeda_rating.index)
memory_based

nama_sepeda,Polygon D7,Polygon Premier 4,Polygon Premier 5,Polygon Siskiu D5,Polygon Siskiu T7,Polygon Siskiu T8,Polygon Xtrada 5,Polygon Xtrada 6.1,Polygon Xtrada 7
nama_sepeda,,,,,,,,,
Polygon D7,1.000000,0.397697,0.499534,0.514582,0.594675,0.455715,0.417434,0.353872,0.587058
Polygon Premier 4,0.397697,1.000000,0.397327,0.610977,0.407586,0.258909,0.356172,0.444719,0.499629
Polygon Premier 5,0.499534,0.397327,1.000000,0.610709,0.630972,0.535397,0.430282,0.594430,0.585517
Polygon Siskiu D5,0.514582,0.610977,0.610709,1.000000,0.611357,0.486645,0.642204,0.602131,0.648748
Polygon Siskiu T7,0.594675,0.407586,0.630972,0.611357,1.000000,0.570285,0.526916,0.485800,0.550329
Polygon Siskiu T8,0.455715,0.258909,0.535397,0.486645,0.570285,1.000000,0.510576,0.552293,0.679274
Polygon Xtrada 5,0.417434,0.356172,0.430282,0.642204,0.526916,0.510576,1.000000,0.716030,0.530395
Polygon Xtrada 6.1,0.353872,0.444719,0.594430,0.602131,0.485800,0.552293,0.716030,1.000000,0.551268
Polygon Xtrada 7,0.587058,0.499629,0.585517,0.648748,0.550329,0.679274,0.530395,0.551268,1.000000


In [ ]:
def predict_bikes (rating, similarity, type='user'):
  if type == 'item' :
    pred = rating.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
  return pred

In [ ]:
bikes_prediction = predict_bikes(train_data_matrix, bikes_similarity, type='item')
bikes_prediction

array([[3.47912772, 3.15336056, 3.75154968, 3.63377801, 3.74825767,
        3.77318419, 3.80569412, 3.81099171, 3.6892325 ],
       [2.07362007, 1.93853946, 2.2018069 , 2.26201224, 1.98199907,
        1.92582781, 2.47409189, 2.47475554, 1.95138681],
       [1.91738091, 1.71977545, 1.96943934, 2.01101062, 2.23211833,
        2.41785842, 2.51638182, 2.33661002, 2.22734923],
       [1.50884723, 1.72042034, 1.52415646, 1.60662249, 1.34950058,
        1.36670726, 1.2804904 , 1.31965774, 1.7228724 ],
       [1.78417962, 1.47632249, 1.57592118, 1.81139406, 1.92566943,
        1.71731329, 1.85818382, 1.56816832, 1.58377333],
       [2.38324631, 2.69771217, 2.82688943, 2.92263953, 2.87989136,
        2.40712825, 3.00723889, 2.88403702, 2.42222295],
       [0.71841864, 1.10847799, 0.8446402 , 0.97027208, 0.80684019,
        0.8434621 , 1.28195956, 1.32769952, 0.85144342],
       [1.61641201, 1.72155853, 1.86100231, 1.95520733, 1.65227608,
        1.80541454, 2.06378568, 2.05633311, 2.01736954],


In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
  prediction = prediction[ground_truth.nonzero()].flatten()
  ground_truth = ground_truth[ground_truth.nonzero()].flatten()
  return sqrt(mean_squared_error(prediction, ground_truth))

In [ ]:
print ('Item Based RMSE = ' + str(rmse(bikes_prediction, test_data_matrix)))

Item Based RMSE = 2.1091126026500593


# **Output**

In [ ]:
def get_memory_based(title, r=3):
    print('Terima kasih membeli sepeda', title, 'kami rekomendasikan anda untuk membeli sepeda:', )
    rf = memory_based[title].sort_values(ascending=False)[:r+1]
    if title in rf:
      del rf[title]
    return rf

In [ ]:
start = time.time()
recommendation = get_memory_based("Polygon Siskiu T7", 2)
end = time.time()
timer = end - start
print(recommendation)
print(timer)

Terima kasih membeli sepeda Polygon Siskiu T7 kami rekomendasikan anda untuk membeli sepeda:
nama_sepeda
Polygon Xtrada 7     0.615515
Polygon Siskiu T8    0.596936
Name: Polygon Siskiu T7, dtype: float64
0.0052487850189208984
